In [1]:
from tabulate import tabulate

class ClassicalRelationalDatabase:
    def __init__(self):
        self.tables = {}

    def add_table(self, name, columns):
        self.tables[name] = columns

    def select(self, table, condition):
        selected_rows = []
        for row in self.tables[table]:
            if condition(row):
                selected_rows.append(row)
        return selected_rows

    def cartesian_product(self, table1, table2):
        cartesian_product = []
        for row1 in self.tables[table1]:
            for row2 in self.tables[table2]:
                cartesian_product.append({**row1, **row2})
        return cartesian_product

    def join(self, table1, table2, common_attribute):
        join_result = []
        for row1 in self.tables[table1]:
            for row2 in self.tables[table2]:
                if row1[common_attribute] == row2[common_attribute]:
                    join_result.append({**row1, **row2})
        return join_result


# Example usage
db = ClassicalRelationalDatabase()

# Define tables
db.add_table("patients", [{"P_NAME": "Emily", "P_AGE": 20, "P_DISEASE": "Diabetes"},
                          {"P_NAME": "James", "P_AGE": 52, "P_DISEASE": "Asthma"},
                          {"P_NAME": "Sarah", "P_AGE": 35, "P_DISEASE": "Hypertension"}])
db.add_table("doctors", [{"D_NAME": "Dr. Smith", "D_COST": 100, "P_DISEASE": "Diabetes"},
                          {"D_NAME": "Dr. Johnson", "D_COST": 900, "P_DISEASE": "Asthma"},
                          {"D_NAME": "Dr. Brown", "D_COST": 300, "P_DISEASE": "Hypertension"}])

# Perform selection
selected_patients = db.select("patients", lambda row: row["P_DISEASE"] == "Diabetes")
print("Selected patients with Diabetes:")
print(tabulate(selected_patients, headers="keys"))

# Perform Cartesian product
cartesian_product = db.cartesian_product("patients", "doctors")
print("\nCartesian product of patients and doctors:")
print(tabulate(cartesian_product, headers="keys"))

# Perform Join
join_result = db.join("patients", "doctors", "P_DISEASE")
print("\nJoin result:")
print(tabulate(join_result, headers="keys"))







Selected patients with Diabetes:
P_NAME      P_AGE  P_DISEASE
--------  -------  -----------
Emily          20  Diabetes

Cartesian product of patients and doctors:
P_NAME      P_AGE  P_DISEASE     D_NAME         D_COST
--------  -------  ------------  -----------  --------
Emily          20  Diabetes      Dr. Smith         100
Emily          20  Asthma        Dr. Johnson       900
Emily          20  Hypertension  Dr. Brown         300
James          52  Diabetes      Dr. Smith         100
James          52  Asthma        Dr. Johnson       900
James          52  Hypertension  Dr. Brown         300
Sarah          35  Diabetes      Dr. Smith         100
Sarah          35  Asthma        Dr. Johnson       900
Sarah          35  Hypertension  Dr. Brown         300

Join result:
P_NAME      P_AGE  P_DISEASE     D_NAME         D_COST
--------  -------  ------------  -----------  --------
Emily          20  Diabetes      Dr. Smith         100
James          52  Asthma        Dr. Johnson       

In [7]:
class FuzzyRelationalDatabase(ClassicalRelationalDatabase):
    def __init__(self):
        super().__init__()

    def fuzzy_select(self, table, condition, membership_function):
        selected_rows = []
        for row in self.tables[table]:
            membership_degree = membership_function(row)
            if condition(row):
                selected_rows.append((row, membership_degree))
        return selected_rows

    def fuzzy_cartesian_product(self, table1, table2, membership_function):
        cartesian_product = []
        for row1 in self.tables[table1]:
            for row2 in self.tables[table2]:
                membership_degree1 = membership_function(row1)
                membership_degree2 = membership_function(row2)
                cartesian_product.append(({**row1, **row2}, min(membership_degree1, membership_degree2)))
        return cartesian_product

    def fuzzy_join(self, table1, table2, common_attribute, membership_function):
        join_result = []
        for row1 in self.tables[table1]:
            for row2 in self.tables[table2]:
                if row1[common_attribute] == row2[common_attribute]:
                    membership_degree1 = membership_function(row1)
                    membership_degree2 = membership_function(row2)
                    join_result.append(({**row1, **row2}, min(membership_degree1, membership_degree2)))
        return join_result

# Example usage
fdb = FuzzyRelationalDatabase()

# Define tables
fdb.add_table("patients", [{"P_NAME": "Emily", "P_AGE": 20, "P_DISEASE": "Diabetes"},
                           {"P_NAME": "James", "P_AGE": 52, "P_DISEASE": "Asthma"},
                           {"P_NAME": "Sarah", "P_AGE": 35, "P_DISEASE": "Hypertension"}])
fdb.add_table("doctors", [{"D_NAME": "Dr. Smith", "D_COST": 100, "P_DISEASE": "Diabetes"},
                           {"D_NAME": "Dr. Johnson", "D_COST": 900, "P_DISEASE": "Asthma"},
                           {"D_NAME": "Dr. Brown", "D_COST": 300, "P_DISEASE": "Hypertension"}])

# Membership function for fuzzy selection
def membership_function(row):
    # Example membership function, can be customized as needed
    if "P_AGE" in row:
        if row["P_AGE"] < 30:
            return 1.0
        elif 30 <= row["P_AGE"] <= 40:
            return 0.7
        else:
            return 0.5 
    return 1.0

# Perform fuzzy selection
fuzzy_selected_patients = fdb.fuzzy_select("patients", lambda row: row["P_DISEASE"] == "Diabetes", membership_function)
print("Fuzzy selected patients with Diabetes:")
print(tabulate(fuzzy_selected_patients, headers=["Patient Details", "Membership Degree"], tablefmt="pretty"))

# Perform fuzzy Cartesian product
fuzzy_cartesian_product = fdb.fuzzy_cartesian_product("patients", "doctors", membership_function)
print("\nFuzzy Cartesian product of patients and doctors:")
print(tabulate(fuzzy_cartesian_product, headers=["Combined Details", "Membership Degree"], tablefmt="pretty"))

# Perform fuzzy join
fuzzy_join_result = fdb.fuzzy_join("patients", "doctors", "P_DISEASE", membership_function)
print("\nFuzzy join result:")
print(tabulate(fuzzy_join_result, headers=["Combined Details", "Membership Degree"], tablefmt="pretty"))


Fuzzy selected patients with Diabetes:
+-----------------------------------------------------------+-------------------+
|                      Patient Details                      | Membership Degree |
+-----------------------------------------------------------+-------------------+
| {'P_NAME': 'Emily', 'P_AGE': 20, 'P_DISEASE': 'Diabetes'} |        1.0        |
+-----------------------------------------------------------+-------------------+

Fuzzy Cartesian product of patients and doctors:
+-----------------------------------------------------------------------------------------------------+-------------------+
|                                          Combined Details                                           | Membership Degree |
+-----------------------------------------------------------------------------------------------------+-------------------+
|   {'P_NAME': 'Emily', 'P_AGE': 20, 'P_DISEASE': 'Diabetes', 'D_NAME': 'Dr. Smith', 'D_COST': 100}   |        1.0        |
|   {